# imports

In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log()

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)




log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')

log(str(a))
log()

# pd_query  






In [ ]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension






# if 'cards' not in globals():
#     cards = pd.read_csv('data/cards.csv')
# cards.q('toughness ´r >2 & <5')
# cards.qi()


# df = qp.get_df()
# df.q(
#     r"""
#     # id ´r ?1  ´n test
#     # name ´m ~ x.upper()
#     # is any ´r is any

#     date of birth / age

#     """,
#     diff=None,
#     inplace=False,
#     verbosity=4,
#     )

df = qp.get_df()
df.qi()



In [ ]:
"""
name
    ´v ~ x.title()
    ´r ?j  ´n contains "j"  ´h notes

is any ´r is any
"""

df['name'] = df['name'].str.title()
mask = df['name'].str.lower().str.contains('j')
df['notes'] = ''
df.loc[mask, 'notes'] = 'contains "j"'

df


## value mask

In [ ]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension




class Symbol:
    def __init__(self, symbol, name, description, unary=None, binary=None, **kwargs):
        self.symbol = symbol
        self.name = name
        self.description = description
        self.unary = unary
        self.binary = binary
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self):
        return f'{self.name} (symbol: "{self.symbol}" description: "{self.description})"'
    
    def __str__(self):
        return f'{self.name} (symbol: "{self.symbol}" description: "{self.description})"'

class Symbols:
    def __init__(self, name, *symbols):
        self.name = name
        self.by_name = {symbol.name: symbol for symbol in symbols}
        self.by_symbol = {symbol.symbol: symbol for symbol in symbols}
        for symbol in symbols:
            setattr(self, symbol.name, symbol)

    def __getitem__(self, key):
        if key in self.by_symbol:
            return self.by_symbol[key]
        elif key in self.by_name:
            return self.by_name[key]
        else:
            log(f'error: symbol "{key}" not found in "{self.name}"', 'Symbols.__getitem__', 3)
            return None

    def __iter__(self):
        return iter(self.by_name.values())

    def __repr__(self):
        return f'{self.name}:\n' + '\n\t'.join([str(val) for key,val in self.by_name.items()])
    
    def __str__(self):
        return f'{self.name}:\n' + '\n\t'.join([str(val) for key,val in self.by_name.items()])


class ChangeSettings:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.CHANGE_SETTINGS
        self.connector = CONNECTORS.RESET
        self.operator = OPERATORS.SET_VERBOSITY
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.operators = [OPERATORS.SET_VERBOSITY, OPERATORS.SET_DIFF, OPERATORS.SET_INPLACE]
        self.verbosity = verbosity

    def __repr__(self):
        return f'CHANGE_SETTINGS:\n\tconnector: {self.connector}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self, verbosity=None):
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)
        
    def apply(self, query_obj):
        operator = self.operator
        value = self.value

        if operator == OPERATORS.SET_VERBOSITY:
            if value in ['0', '1', '2', '3', '4', '5']:
                query_obj.verbosity = int(value)
                for instruction in query_obj.instructions:
                    instruction.verbosity = query_obj.verbosity
            else:
                log(f'warning: verbosity must be an integer between 0 and 5. "{value}" is not valid',
                    'df.q()', query_obj.verbosity)
        
        elif operator == OPERATORS.SET_DIFF:
            if value.lower() in ['none', '0']:
                query_obj.diff = None
            elif value.lower() in ['mix', 'new', 'old', 'new+']:
                query_obj.diff = value.lower()
            else:
                log(f'warning: diff must be one of [None, "mix", "old", "new", "new+"]. "{value}" is not valid',
                    'df.q()', query_obj.verbosity)
                
        elif operator == OPERATORS.SET_INPLACE:
            if value.lower() in ['true', '1', 'yes']:
                query_obj.inplace = True
                query_obj.df = query_obj.df_og 
                query_obj.df.qp = query_obj.df_og.qp

            elif value.lower() in ['false', '0', 'no']:
                query_obj.inplace = False
                query_obj.df = None
            else:
                log(f'warning: inplace must be a boolean. "{value}" is not valid',
                    'df.q()', query_obj.verbosity)
    

class SelectCols:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.SELECT_COLS
        self.connector = CONNECTORS.RESET
        self.negation = NEGATION.FALSE
        self.operator = OPERATORS.EQUAL
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.negations = [NEGATION.TRUE]
        self.operators = [
            #binary
            OPERATORS.BIGGER_EQUAL, OPERATORS.SMALLER_EQUAL, OPERATORS.BIGGER, OPERATORS.SMALLER,
            OPERATORS.STRICT_EQUAL, OPERATORS.EQUAL,
            OPERATORS.STRICT_CONTAINS, OPERATORS.CONTAINS,
            OPERATORS.MATCHES_REGEX, OPERATORS.CONTAINS_REGEX,
            OPERATORS.EVAL,
            OPERATORS.LOAD_SELECTION,
        
            #unary
            OPERATORS.IS_ANY,
            OPERATORS.IS_UNIQUE,
            OPERATORS.IS_NA, OPERATORS.IS_NK,
            OPERATORS.IS_STR, OPERATORS.IS_INT, OPERATORS.IS_FLOAT, OPERATORS.IS_NUM, OPERATORS.IS_BOOL,
            OPERATORS.IS_DATE, OPERATORS.IS_DATETIME,
            OPERATORS.IS_YN, OPERATORS.IS_YES, OPERATORS.IS_NO,
            ]
        self.verbosity = verbosity
             
    def __repr__(self):
        return f'SELECT_COLS:\n\tconnector: {self.connector}\n\tnegation: {self.negation}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self):
        #parse the expression
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.negation, text = match_symbol(text, self.negation, self.negations, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        if self.operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{self.operator}" cannot use a value. value "{self.value}" will be ignored',
                '_parse_expression', self.verbosity)
            self.value = ''

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)

    def apply(self, query_obj):
        if query_obj.df is None:
            df = query_obj.df_og
        else:
            df = query_obj.df

        cols = df.columns.to_series()

        cols_filtered_new = filter_series(query_obj, cols, instruction=self)

        if cols_filtered_new.any() == False:
            log(f'warning: no columns fulfill the condition in "{self.text}"',
                'df.q()', self.verbosity)

        query_obj.cols_filtered = _update_index(query_obj.cols_filtered, cols_filtered_new, self.connector)


class SelectRows:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.SELECT_ROWS
        self.connector = CONNECTORS.RESET
        self.scope = SCOPE.ANY  #only for rows
        self.negation = NEGATION.FALSE
        self.operator = OPERATORS.EQUAL
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.scopes = [SCOPE.ANY, SCOPE.ALL, SCOPE.INDEX]  #only for rows
        self.negations = [NEGATION.TRUE]
        self.operators = [
            #binary
            OPERATORS.BIGGER_EQUAL, OPERATORS.SMALLER_EQUAL, OPERATORS.BIGGER, OPERATORS.SMALLER,
            OPERATORS.STRICT_EQUAL, OPERATORS.EQUAL,
            OPERATORS.STRICT_CONTAINS, OPERATORS.CONTAINS,
            OPERATORS.MATCHES_REGEX, OPERATORS.CONTAINS_REGEX,
            OPERATORS.EVAL, OPERATORS.COL_EVAL,  #only for rows
            OPERATORS.LOAD_SELECTION,
        
            #unary
            OPERATORS.IS_ANY,
            OPERATORS.IS_UNIQUE, OPERATORS.IS_FIRST, OPERATORS.IS_LAST,
            OPERATORS.IS_NA, OPERATORS.IS_NK,
            OPERATORS.IS_STR, OPERATORS.IS_INT, OPERATORS.IS_FLOAT, OPERATORS.IS_NUM, OPERATORS.IS_BOOL,
            OPERATORS.IS_DATE, OPERATORS.IS_DATETIME,
            OPERATORS.IS_YN, OPERATORS.IS_YES, OPERATORS.IS_NO,
            ]
        self.verbosity = verbosity
             
    def __repr__(self):
        return f'SELECT_ROWS:\n\tconnector: {self.connector}\n\tscope: {self.scope}\n\tnegation: {self.negation}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self):
        #parse the expression
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.scope, text = match_symbol(text, self.scope, self.scopes, self.verbosity)
        self.negation, text = match_symbol(text, self.negation, self.negations, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        if self.operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{self.operator}" cannot use a value. value "{self.value}" will be ignored',
                '_parse_expression', self.verbosity)
            self.value = ''

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)

    def apply(self, query_obj):
        if query_obj.df is None:
            df = query_obj.df_og
        else:
            df = query_obj.df

        #select rows using parsed expression
        scope = self.scope
        connector = self.connector
        verbosity = query_obj.verbosity

        rows = df.index.to_series()
        cols_filtered = query_obj.cols_filtered

        if cols_filtered.any() == False:
            log(f'warning: row filter cannot be applied when no columns where selected', 'df.q', verbosity)
            return
                
        if scope == SCOPE.INDEX:
            rows_filtered_new = filter_series(query_obj, rows, instruction=self)
            query_obj.rows_filtered = _update_index(query_obj.rows_filtered, rows_filtered_new, connector)

        else:
            rows_filtered_temp = None
            for col in df.columns[cols_filtered]:
                rows_filtered_new = filter_series(query_obj, df[col], instruction=self)
                rows_filtered_temp = _update_index(rows_filtered_temp, rows_filtered_new, scope)
            query_obj.rows_filtered = _update_index(query_obj.rows_filtered, rows_filtered_temp, connector)

            if rows_filtered_temp.any() == False:
                log(f'warning: no rows fulfill the condition in "{self.text}"', 'df.q', verbosity)


class ModifyVals:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.MODIFY_VALS
        self.connector = CONNECTORS.RESET
        self.operator = OPERATORS.SET_VAL
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.operators = [
            OPERATORS.SET_VAL, OPERATORS.ADD_VAL,
            OPERATORS.SET_EVAL, OPERATORS.SET_COL_EVAL,
            OPERATORS.TO_STR, OPERATORS.TO_INT, OPERATORS.TO_FLOAT, OPERATORS.TO_NUM, OPERATORS.TO_BOOL,
            OPERATORS.TO_DATE, OPERATORS.TO_DATETIME, OPERATORS.TO_NA, OPERATORS.TO_NK, OPERATORS.TO_YN,
            ]
        self.verbosity = verbosity

    def __repr__(self):
        return f'MODIFY_VALS:\n\tconnector: {self.connector}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self):
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        if self.operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{self.operator}" cannot use a value. value "{self.value}" will be ignored',
                '_parse_expression', self.verbosity)
            self.value = ''

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)
        
    def apply(self, query_obj):
        if query_obj.df is None:
            query_obj.df = query_obj.df_og.copy()  #default is inplace=False
            query_obj.df.qp = query_obj.df.qp
        
        rows = query_obj.rows_filtered
        cols = query_obj.cols_filtered

        operator = self.operator
        value = self.value

        #data modification  
        if operator == OPERATORS.SET_VAL:
            query_obj.df.loc[rows, cols] = value
        elif operator == OPERATORS.ADD_VAL:
            query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].astype(str) + value
        elif operator == OPERATORS.SET_COL_EVAL:
            query_obj.df.loc[:, cols] = query_obj.df.loc[:, cols].apply(lambda x: eval(value, {'col': x, 'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp}), axis=0)


        elif pd.__version__ >= '2.1.0':  #map was called applymap before 2.1.0
            #data modification
            if operator == OPERATORS.SET_EVAL:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(lambda x: eval(value, {'x': x, 'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp}))


            #type conversion
            elif operator == OPERATORS.TO_STR:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(str)
            elif operator == OPERATORS.TO_INT:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_int)
            elif operator == OPERATORS.TO_FLOAT:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_float)
            elif operator == OPERATORS.TO_NUM:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_num)
            elif operator == OPERATORS.TO_BOOL:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_bool)
            
            elif operator == OPERATORS.TO_DATETIME:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_datetime)
            elif operator == OPERATORS.TO_DATE:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_date)

            elif operator == OPERATORS.TO_NA:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_na)
            elif operator == OPERATORS.TO_NK:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_nk)
            elif operator == OPERATORS.TO_YN:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].map(_yn)

        else:
            #data modification
            if operator == OPERATORS.SET_EVAL:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(lambda x: eval(value, {'x': x, 'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp}))

            #type conversion
            elif operator == OPERATORS.TO_STR:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(str)
            elif operator == OPERATORS.TO_INT:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_int)
            elif operator == OPERATORS.TO_FLOAT:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_float)
            elif operator == OPERATORS.TO_NUM:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_num)
            elif operator == OPERATORS.TO_BOOL:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_bool)
            
            elif operator == OPERATORS.TO_DATETIME:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_datetime)
            elif operator == OPERATORS.TO_DATE:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_date)

            elif operator == OPERATORS.TO_NA:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_na)
            elif operator == OPERATORS.TO_NK:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_nk)
            elif operator == OPERATORS.TO_YN:
                query_obj.df.loc[rows, cols] = query_obj.df.loc[rows, cols].applymap(_yn)


class ModifyHeaders:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.MODIFY_HEADERS
        self.connector = CONNECTORS.RESET
        self.operator = OPERATORS.SET_VAL
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.operators = [
            OPERATORS.SET_VAL, OPERATORS.ADD_VAL,
            OPERATORS.SET_EVAL,
            ]
        self.verbosity = verbosity

    def __repr__(self):
        return f'MODIFY_HEADERS:\n\tconnector: {self.connector}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self):
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        if self.operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{self.operator}" cannot use a value. value "{self.value}" will be ignored',
                '_parse_expression', self.verbosity)
            self.value = ''

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)
        
    def apply(self, query_obj):
        if query_obj.df is None:
            query_obj.df = query_obj.df_og.copy()  #default is inplace=False
            query_obj.df.qp = query_obj.df.qp
        
        cols = query_obj.cols_filtered

        operator = self.operator
        value = self.value


        if operator == OPERATORS.SET_VAL:
            query_obj.df.rename(columns={col: value for col in query_obj.df.columns[cols]}, inplace=True)
            query_obj.cols_filtered.index = query_obj.df.columns
        
        if pd.__version__ >= '2.1.0':
            if operator == OPERATORS.SET_EVAL:
                new_headers = query_obj.df.columns.map(lambda x: eval(value, {'header': x, 'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp}))
                query_obj.df.columns = new_headers
                query_obj.cols_filtered.index = new_headers
        else:
            if operator == OPERATORS.SET_EVAL:
                new_headers = query_obj.df.columns.applymap(lambda x: eval(value, {'header': x, 'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp}))
                query_obj.df.columns = new_headers
                query_obj.cols_filtered.index = new_headers


class NewCol:
    def __init__(self, text=None, linenum=None, verbosity=3):
        self.text = text
        self.linenum = linenum

        #default values
        self.type = TYPES.NEW_COL
        self.connector = CONNECTORS.RESET
        self.operator = OPERATORS.STR_COL
        self.value = ''

        #possible values (omitting those without a symbol)
        self.connectors = [CONNECTORS.AND, CONNECTORS.OR]
        self.operators = [
            OPERATORS.STR_COL,
            OPERATORS.EVAL_COL,
            OPERATORS.SAVE_SELECTION,
            ]
        self.verbosity = verbosity
    
    def __repr__(self):
        return f'NEW_COL:\n\tconnector: {self.connector}\n\toperator: {self.operator}\n\tvalue: {self.value}'
    
    def parse(self):
        self.connector, text = match_symbol(self.text[2:], self.connector, self.connectors, self.verbosity)
        self.operator, text = match_symbol(text, self.operator, self.operators, self.verbosity)
        self.value = text.strip()

        if self.operator.unary and len(self.value)>0:
            log(f'warning: unary operator "{self.operator}" cannot use a value. value "{self.value}" will be ignored',
                '_parse_expression', self.verbosity)
            self.value = ''

        log(f'debug: parsed "{self.text}" as instruction: {self}',
            'df.q()', self.verbosity)
        
    def apply(self, query_obj):
        if query_obj.df is None:
            query_obj.df = query_obj.df_og.copy()  #default is inplace=False
            query_obj.df.qp = query_obj.df.qp
        
        if self.operator == OPERATORS.STR_COL:
            for i in range(1, 1001):
                if i == 1000:
                    log(f'warning: could not add new column. too many columns named "new<x>"',
                        'df.q.new_col', query_obj.verbosity)
                    break

                header = 'new' + str(i)
                if header not in query_obj.df.columns:
                    query_obj.df[header] = ''
                    query_obj.df.loc[query_obj.rows_filtered, header] = self.value
                    query_obj.cols_filtered = pd.Index([True if col == header else False for col in query_obj.df.columns])
                    query_obj.cols_filtered.index = query_obj.df.columns
                    break
        
        elif self.operator == OPERATORS.EVAL_COL:
            for i in range(1, 1001):
                if i == 1000:
                    log(f'warning: could not add new column. too many columns named "new<x>"',
                        'df.q.new_col', query_obj.verbosity)
                    break

                header = 'new' + str(i)
                if header not in query_obj.df.columns:
                    query_obj.df[header] = pd.NA
                    query_obj.df.loc[query_obj.rows_filtered, header] = eval(self.value, {'df': query_obj.df, 'pd': pd, 'np': np, 'qp': qp})
                    query_obj.cols_filtered = pd.Index([True if col == header else False for col in query_obj.df.columns])
                    query_obj.cols_filtered.index = query_obj.df.columns
                    break
        

        elif self.operator == OPERATORS.SAVE_SELECTION:
            if self.value in query_obj.df.columns:
                log(f'warning: column "{self.value}" already exists in dataframe. selecting existing col and resetting values',
                    'df.q.new_col', query_obj.verbosity)
                query_obj.df[self.value] = query_obj.rows_filtered
                query_obj.cols_filtered = pd.Index([True if col == self.value else False for col in query_obj.df.columns])
                query_obj.cols_filtered.index = query_obj.df.columns
            else:
                query_obj.df[self.value] = query_obj.rows_filtered
                query_obj.cols_filtered = pd.Index([True if col == self.value else False for col in query_obj.df.columns])
                query_obj.cols_filtered.index = query_obj.df.columns



COMMENT = Symbol('#', 'COMMENT', 'comments out the rest of the line')
ESCAPE = Symbol('`', 'ESCAPE', 'escape the next character')

TYPES = Symbols('TYPES',
    Symbol('´s', 'CHANGE_SETTINGS', 'change query settings', instruction=ChangeSettings),
    Symbol('´c', 'SELECT_COLS', 'select columns fulfilling a condition', instruction=SelectCols),
    Symbol('´r', 'SELECT_ROWS', 'select rows fulfilling a condition', instruction=SelectRows),
    Symbol('´v', 'MODIFY_VALS', 'modify the selected values', instruction=ModifyVals),
    Symbol('´h', 'MODIFY_HEADERS', 'modify headers of the selected columns', instruction=ModifyHeaders),
    Symbol('´n', 'NEW_COL', 'add new column', instruction=NewCol),
    )

CONNECTORS = Symbols('CONNECTORS',
    Symbol('', 'RESET', 'only the current condition must be fulfilled'),
    Symbol('&', 'AND', 'this condition and the previous condition/s must be fulfilled'),
    Symbol('/', 'OR', 'this condition or the previous condition/s must be fulfilled'),
    )

SCOPE = Symbols('SCOPE',
    Symbol('any', 'ANY', 'any of the currently selected columns must fulfill the condition'),
    Symbol('all', 'ALL', 'all of the currently selected columns must fulfill the condition'),
    Symbol('idx', 'INDEX', 'the index of the dataframe must fulfill the condition'),
    )

NEGATION = Symbols('NEGATION',
    Symbol('', 'FALSE', 'dont negate the condition'),
    Symbol('!', 'TRUE', 'negate the condition'),
    )

OPERATORS = Symbols('OPERATORS',
    #for changing settings
    Symbol('verbosity=', 'SET_VERBOSITY', 'change the verbosity level'),
    Symbol('diff=', 'SET_DIFF', 'change the diff setting'),
    Symbol('inplace=', 'SET_INPLACE', 'change the inplace setting'),


    #for filtering
    Symbol('>=', 'BIGGER_EQUAL', 'bigger or equal', binary=True),
    Symbol('<=', 'SMALLER_EQUAL', 'smaller or equal', binary=True),
    Symbol('>', 'BIGGER', 'bigger', binary=True),
    Symbol('<', 'SMALLER', 'smaller', binary=True),

    Symbol('==', 'STRICT_EQUAL', 'equal to (case sensitive)', binary=True),
    Symbol('=', 'EQUAL', 'equal to', binary=True),

    Symbol('??', 'STRICT_CONTAINS', 'contains a string (case sensitive)', binary=True),
    Symbol('?', 'CONTAINS', 'contains a string (not case sensitive)', binary=True),

    Symbol('r=', 'MATCHES_REGEX', 'matches a regex', binary=True),
    Symbol('r?', 'CONTAINS_REGEX', 'contains a regex', binary=True),

    Symbol('~', 'EVAL', 'select values by evaluating a python expression on each value', binary=True),
    Symbol('col~', 'COL_EVAL', 'select rows by evaluating a python expression on a whole column', binary=True),

    Symbol('@', 'LOAD_SELECTION', 'load a saved selection from a boolean column', binary=True),

    Symbol('is any', 'IS_ANY', 'is any value', unary=True),
    Symbol('is str', 'IS_STR', 'is string', unary=True),
    Symbol('is int', 'IS_INT', 'is integer', unary=True),
    Symbol('is float', 'IS_FLOAT', 'is float', unary=True),
    Symbol('is num', 'IS_NUM', 'is number', unary=True),
    Symbol('is bool', 'IS_BOOL', 'is boolean', unary=True),
    Symbol('is datetime', 'IS_DATETIME', 'is datetime', unary=True),
    Symbol('is date', 'IS_DATE', 'is date', unary=True),
    Symbol('is na', 'IS_NA', 'is missing value', unary=True),
    Symbol('is nk', 'IS_NK', 'is not known value', unary=True),
    Symbol('is yn', 'IS_YN', 'is yes or no value', unary=True),
    Symbol('is yes', 'IS_YES', 'is yes value', unary=True),
    Symbol('is no', 'IS_NO', 'is no value', unary=True),
    Symbol('is unique', 'IS_UNIQUE', 'is a unique value', unary=True),
    Symbol('is first', 'IS_FIRST', 'is the first value (of multiple values)', unary=True),
    Symbol('is last', 'IS_LAST', 'is the last value (of multiple values)', unary=True),


    #for modifying values and headers
    Symbol('=', 'SET_VAL', 'convert to string'),
    Symbol('+=', 'ADD_VAL', 'append to string (coerce to string if needed)'),

    Symbol('~', 'SET_EVAL', 'convert by evaluating a python expression for each selected value/header'),
    Symbol('col~', 'SET_COL_EVAL', 'convert by evaluating a python expression for each selected column'),

    Symbol('to str', 'TO_STR', 'convert to string', unary=True),
    Symbol('to int', 'TO_INT', 'convert to integer', unary=True),
    Symbol('to float', 'TO_FLOAT', 'convert to float', unary=True),
    Symbol('to num', 'TO_NUM', 'convert to number', unary=True),
    Symbol('to bool', 'TO_BOOL', 'convert to boolean', unary=True),
    Symbol('to datetime', 'TO_DATETIME', 'convert to datetime', unary=True),
    Symbol('to date', 'TO_DATE', 'convert to date', unary=True),
    Symbol('to na', 'TO_NA', 'convert to missing value', unary=True),
    Symbol('to nk', 'TO_NK', 'convert to not known value', unary=True),
    Symbol('to yn', 'TO_YN', 'convert to yes or no value', unary=True),


    #for adding new columns
    Symbol('=', 'STR_COL', 'add new column, fill it with the given string and select it'),
    Symbol('~', 'EVAL_COL', 'add new column, fill it by evaluating a python expression and select it'),
    Symbol('@', 'SAVE_SELECTION', 'add a new boolean column with the given name and select it. all currently selected rows are set to True, the rest to False'),
    )



def tokenize(code, verbosity=3):
    lines = []
    instructions = []

    #get lines and instruction blocks
    for line_num, line in enumerate(code.split('\n')):
        line = line.strip()
        lines.append([line_num, line])
        line = line.split(COMMENT.symbol)[0].strip()
    
        if line == '':
            continue


        escape = False
        chars_in_instruction = 0
        instruction_type = TYPES.SELECT_COLS.symbol  #default

        for i, char in enumerate(line):
            if escape:
                instructions[-1].text += char
                chars_in_instruction += 1
                escape = False
                continue
            elif char == ESCAPE.symbol:
                escape = True
                continue

            if char == '´':
                instruction_type = char + line[i+1]
                instructions.append(TYPES[instruction_type].instruction(char, line_num, verbosity))
                chars_in_instruction = 1
            elif char in [CONNECTORS.AND.symbol, CONNECTORS.OR.symbol]:
                if chars_in_instruction >= 3:
                    instructions.append(TYPES[instruction_type].instruction(f'{instruction_type} {char}', line_num, verbosity))
                    chars_in_instruction = 3
                elif i == 0:
                    instructions.append(TYPES[instruction_type].instruction(f'{instruction_type} {char}', line_num, verbosity))
                    chars_in_instruction = 3
                else:
                    instructions[-1].text += char
                    chars_in_instruction += 1
            elif i == 0:
                instructions.append(TYPES[instruction_type].instruction(f'{instruction_type} {char}', line_num, verbosity))
                chars_in_instruction = 3
            elif char == ' ':
                instructions[-1].text += char
            else:
                instructions[-1].text += char
                chars_in_instruction += 1

    return lines, instructions


def match_symbol(string, default, symbols, verbosity):
    string = string.strip()

    for symbol in symbols:
        if string.startswith(symbol.symbol):
            log(f'trace: found symbol "{symbol}" in string "{string}"', 'match_symbol', verbosity)
            return symbol, string[len(symbol.symbol):].strip()
    
    log(f'trace: no symbol found in string "{string}". using default "{default}"', 'match_symbol', verbosity)
    
    if default is None:
        return None, string
    if string.startswith(default.symbol):
        string = string[len(default.symbol):].strip()
    return default, string


def filter_series(query_obj, series, instruction):
    negation = instruction.negation
    operator = instruction.operator
    value = instruction.value
    verbosity = instruction.verbosity
    df = query_obj.df


    #numeric comparison
    if operator == OPERATORS.BIGGER_EQUAL:
        filtered = pd.to_numeric(series, errors='coerce') >= pd.to_numeric(value)
    elif operator == OPERATORS.SMALLER_EQUAL:
        filtered = pd.to_numeric(series, errors='coerce') <= pd.to_numeric(value)
    elif operator == OPERATORS.BIGGER:
        filtered = pd.to_numeric(series, errors='coerce') > pd.to_numeric(value)
    elif operator == OPERATORS.SMALLER:
        filtered = pd.to_numeric(series, errors='coerce') < pd.to_numeric(value)


    #string equality comparison
    elif operator == OPERATORS.STRICT_EQUAL:
        filtered = series.astype(str) == value
    elif operator == OPERATORS.EQUAL:
        value_lenient = [value]
        try:
            value_lenient.append(str(float(value)))
            value_lenient.append(str(int(float(value))))
        except:
            value_lenient.append(value.lower())
        filtered = series.astype(str).str.lower().isin(value_lenient)


    #substring comparison
    elif operator == OPERATORS.STRICT_CONTAINS:
        filtered = series.astype(str).str.contains(value, case=True, regex=False)
    elif operator == OPERATORS.CONTAINS:
        filtered = series.astype(str).str.contains(value, case=False, regex=False)


    #regex comparison
    elif operator == OPERATORS.MATCHES_REGEX:
        filtered = series.astype(str).str.fullmatch(value) 
    elif operator == OPERATORS.CONTAINS_REGEX:
        filtered = series.astype(str).str.contains(value)


    #lambda function
    elif operator == OPERATORS.EVAL:
        filtered = series.apply(lambda x: eval(value, {'x': x, 'col': series, 'df': df, 'pd': pd, 'np': np, 'qp': qp}))
    elif operator == OPERATORS.COL_EVAL:
        filtered = eval(value, {'col': series, 'df': df, 'pd': pd, 'np': np, 'qp': qp})

    #load saved selection
    elif operator == OPERATORS.LOAD_SELECTION:
        if value in df.columns:
            filtered = df[value]
        else:
            log(f'error: column "{value}" does not exist in dataframe. cannot load selection',
                '_filter()', verbosity)


    #type checks
    elif operator == OPERATORS.IS_STR:
        filtered = series.apply(lambda x: isinstance(x, str))
    elif operator == OPERATORS.IS_INT:
        filtered = series.apply(lambda x: isinstance(x, int))
    elif operator == OPERATORS.IS_FLOAT:
        filtered = series.apply(lambda x: isinstance(x, float))
    elif operator == OPERATORS.IS_NUM:
        filtered = series.apply(lambda x: _num(x, errors='ERROR')) != 'ERROR'
    elif operator == OPERATORS.IS_BOOL:
        filtered = series.apply(lambda x: isinstance(x, bool))

    elif operator == OPERATORS.IS_DATETIME:
        filtered = series.apply(lambda x: _datetime(x, errors='ERROR')) != 'ERROR'
    elif operator == OPERATORS.IS_DATE:
        filtered = series.apply(lambda x: _date(x, errors='ERROR')) != 'ERROR'

    elif operator == OPERATORS.IS_ANY:
        filtered = series.apply(lambda x: True)
    elif operator == OPERATORS.IS_NA:
        filtered = series.apply(lambda x: _na(x, errors='ERROR')) != 'ERROR'
    elif operator == OPERATORS.IS_NK:
        filtered = series.apply(lambda x: _nk(x, errors='ERROR')) != 'ERROR'
    elif operator == OPERATORS.IS_YN:
        filtered = series.apply(lambda x: _yn(x, errors='ERROR')) != 'ERROR'
    elif operator == OPERATORS.IS_YES:
        filtered = series.apply(lambda x: _yn(x, errors='ERROR', yes=1)) == 1
    elif operator == OPERATORS.IS_NO:
        filtered = series.apply(lambda x: _yn(x, errors='ERROR', no=0)) == 0
        
    elif operator == OPERATORS.IS_UNIQUE:
        filtered = series.duplicated(keep=False) == False
    elif operator == OPERATORS.IS_FIRST:
        filtered = series.duplicated(keep='first') == False
    elif operator == OPERATORS.IS_LAST:
        filtered = series.duplicated(keep='last') == False

    else:
        log(f'error: operator "{operator}" is not implemented', '_filter()', verbosity)
        filtered = None


    if negation == NEGATION.TRUE:
        filtered = ~filtered

    return filtered


def _update_index(values, values_new, connector):
    if values is None:
        values = values_new
    elif connector == CONNECTORS.RESET:
        values = values_new
    elif connector in [CONNECTORS.AND, SCOPE.ALL]:
        values &= values_new
    elif connector in [CONNECTORS.OR, SCOPE.ANY]:
        values |= values_new
    return values


@pd.api.extensions.register_dataframe_accessor('q')
class DataFrameQuery:
    """
    wip
    """

    def __init__(self, df: pd.DataFrame):
        _check_df(df)
        self.df_og = df
        self.mask = pd.DataFrame(True, index=df.index, columns=df.columns)

    def __repr__(self):
        return 'docstring of dataframe accessor pd_object.q():' + self.__doc__
    
    def __call__(self,
            code='',  #code in string form for filtering and modifying data
            inplace=False,  #make modifications inplace or just return a new dataframe.
            verbosity=3,  #verbosity level for logging. 0: no logging, 1: errors, 2: warnings, 3: info, 4: debug
            diff=None,  #[None, 'mix', 'old', 'new', 'new+']
            diff_max_cols=200,  #maximum number of columns to display when using diff. None: show all
            diff_max_rows=20,  #maximum number of rows to display when using diff. None: show all
            ):
        
        #setup

        self.code = code
        self.inplace = inplace
        self.verbosity = verbosity
        self.diff = diff
        self.diff_max_cols = diff_max_cols
        self.diff_max_rows = diff_max_rows

        if inplace is False:
            self.df = None
        else:
            self.df = self.df_og 
            self.df.qp = self.df_og.qp 

        self.cols_filtered = pd.Index([True for col in self.df_og.columns])
        self.rows_filtered = pd.Index([True for row in self.df_og.index])



        #instructions

        self.lines, self.instructions = tokenize(self.code, self.verbosity)

        for instruction in self.instructions:
            instruction.parse()
            instruction.apply(self)

   
        #results
        if self.df is None:
            df = self.df_og
        else:
            df = self.df

        self.df_filtered = df.loc[self.rows_filtered, self.cols_filtered]
        self.df_filtered.qp = df.qp
        self.df_filtered.qp.code = self.code
    
        if self.diff is None:
            return self.df_filtered 
        else:
            #show difference before and after filtering

            if 'meta' in df.columns and 'meta' not in self.df_filtered.columns:
                self.df_filtered.insert(0, 'meta', df.loc[self.rows_filtered, 'meta'])

            result = _diff(
                self.df_filtered, df, mode=self.diff,
                max_cols=self.diff_max_cols, max_rows=self.diff_max_rows,
                verbosity=self.verbosity)  
            return  result  
   


@pd.api.extensions.register_dataframe_accessor('qi')
class DataFrameQueryInteractiveMode:
    """
    Wrapper for df.q() for interactive use in Jupyter notebooks.
    """
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __call__(self):
        kwargs = {'df': fixed(self.df)}

        #code input
        ui_code = widgets.Textarea(
            value='',
            placeholder='Enter query code here',
            layout=Layout(height='95%')
            )


        #query builder

        instruction = TYPES.SELECT_COLS.instruction()

        i_type = widgets.Dropdown(
            options=[(s.description, s.symbol) for s in TYPES],
            value=instruction.type.symbol,
            )
        
        i_scope = widgets.Dropdown(
            disabled=True,
            options=[''],
            value='',
            )

        i_negate = widgets.ToggleButtons(
            options=[('dont negate condition', ''), ('negate condition', '!')],
            value='',
            )

        i_operator = widgets.Dropdown(
            options=[(s.description, s.symbol) for s in instruction.operators],
            value=instruction.operator.symbol,
            )
        
        i_value = widgets.Text(
            value='',
            )
        

        i_text = widgets.Text(
            value=f'\n{i_type.value} {i_scope.value} {i_negate.value}{i_operator.value} {i_value.value}',
            disabled=True,
            )
        

        def update_options(*args):
            instruction = TYPES[i_type.value].instruction()

            if hasattr(instruction, 'scopes'):
                i_scope.disabled = False
                i_scope.options = [(s.description, s.symbol) for s in instruction.scopes]
            else:
                i_scope.disabled = True
                i_scope.options = ['']

            if hasattr(instruction, 'negations'):
                i_negate.disabled = False
                i_negate.options = [('dont negate condition', ''), ('negate condition', '!')]
            else:
                i_negate.disabled = True
                i_negate.options = ['', '']

            i_operator.options = [(s.description, s.symbol) for s in instruction.operators]
            i_operator.value = instruction.operator.symbol

        def update_text(*args):
            i_text.value = f'{i_type.value} {i_scope.value} {i_negate.value}{i_operator.value} {i_value.value}'

        i_type.observe(update_options, 'value')
        i_type.observe(update_text, 'value')
        i_scope.observe(update_text, 'value')
        i_negate.observe(update_text, 'value')
        i_operator.observe(update_text, 'value')
        i_value.observe(update_text, 'value')

        
        ui_add_instruction = widgets.Button(
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='adds the selected instruction to the query code',
            icon='check' # (FontAwesome names without the `fa-` prefix)
            )

        def add_instruction(ui_code, i_text):
            if i_text.value.startswith('´c'):
                ui_code.value += f'\n{i_text.value}'
            else:
                ui_code.value += f'   {i_text.value}'

        ui_add_instruction.on_click(lambda b: add_instruction(ui_code, i_text))

        kwargs['code'] = ui_code


        ui_diff = widgets.ToggleButtons(
            options=[None, 'mix', 'old', 'new', 'new+'],
            description='show differences mode:',
            tooltips=[
                'dont show differences, just show the new (filtered) dataframe.',
                'show new (filtered) dataframe plus all the removed (filtered) values from the old dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show old (unfiltered) dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show new (filtered) dataframe. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                'show new (filtered) dataframe but also adds metadata columns with the prefix "#". If a value changed, the metadata column contains the old value. values affected by the filters are marked green (newly added), yellow (modified), red (deleted)',
                ],
            )
        kwargs['diff'] = ui_diff

        ui_verbosity = widgets.ToggleButtons(
            options=[0, 1, 2, 3, 4, 5],
            value=3,
            description='verbosity level:',
            tooltips=[
                'no logging',
                'only errors',
                'errors and warnings',
                'errors, warnings and info',
                'errors, warnings, info and debug',
                'errors, warnings, info, debug and trace',
                ],
            )
        
        kwargs['verbosity'] = ui_verbosity

        ui_inplace = widgets.ToggleButtons(
            options=[True, False],
            value=False,
            description='make modifications inplace:',
            tooltips=[
                'make modifications inplace, e.g. change the original dataframe.',
                'return a new dataframe with the modifications. lower performance.',
                ],
            )
        kwargs['inplace'] = ui_inplace


        ui_settings = VBox([
            ui_diff,
            ui_verbosity,
            ui_inplace,
            ])
        
        
        #some general info and statistics about the df
        ui_details = widgets.HTML(
            value=f"""
            <b>shape:</b> {self.df.shape}<br>
            <b>memory usage:</b> {self.df.memory_usage().sum()} bytes<br>
            <b>unique values:</b> {self.df.nunique().sum()}<br>
            <b>missing values:</b> {self.df.isna().sum().sum()}<br>
            <b>columns:</b><br> {'<br>'.join([f'{col} ({dtype})' for col, dtype in list(zip(self.df.columns, self.df.dtypes))])}<br>
            """
            )
        

        ui_info = widgets.Tab(
            children=[
                ui_settings,
                ui_details,
                widgets.HTML(value=DataFrameQuery.__doc__),
                ],
            titles=['settings', 'details', 'readme'],
            layout=Layout(width='30%', height='95%')
            )
        

        ui_input = VBox([
            widgets.HTML(value='<b>query builder:</b>'),
            i_text,
            i_type,
            i_scope,
            i_negate,
            i_operator,
            i_value,
            ui_add_instruction,
            ])
        
        # ui_input = HBox([ui_code, ui_instruction_builder], layout=Layout(width='50%', height='100%'))
        ui = HBox([ui_code, ui_input, ui_info], layout=Layout(width='100%', height='300px'))

        display(ui)
        out = HBox([interactive_output(_interactive_mode, kwargs)], layout=Layout(overflow_y='auto'))
        display(out)


def _interactive_mode(**kwargs):

    df = kwargs.pop('df')

    result = df.q(
        code=kwargs['code'],
        inplace=kwargs['inplace'],
        diff=kwargs['diff'],
        verbosity=kwargs['verbosity'],
        # max_cols=kwargs['max_cols'],
        # max_rows=kwargs['max_rows'],
        )
    
    display(result)
    return result 





# if 'cards' not in globals():
#     cards = pd.read_csv('data/cards.csv')
# cards.q('toughness ´r >2 & <5')
# cards.qi()


# df = qp.get_df()
# df.q(
#     r"""
#     # id ´r ?1  ´n test
#     # name ´m ~ x.upper()
#     # is any ´r is any

#     date of birth / age

#     """,
#     diff=None,
#     inplace=False,
#     verbosity=4,
#     )

df = qp.get_df()
df.qi()



In [ ]:

df = pd.DataFrame({
    'a': [0,1,2],
    'b': [-1, 0, 1],
    'c': [0.1, 0.2, 0.3],
    })

m0 = pd.DataFrame(True, index=df.index, columns=df.columns)

m1 = df > 0
m2 = df['b'] < 0 
m3 = df[1:] > 0

# df[m1]
# df[m1].dropna(how='all')

m1 | m2


In [ ]:

cards = pd.read_csv('data/cards.csv').fillna('')
c = cards.q('power / toughness ´v to num  ´c is any  ´r is any')


In [ ]:
m = pd.DataFrame(True, index=c.index, columns=c.columns)


m1 = c['power'] > 10
m2 = c['toughness'] > 10
m3 = m1 | m2


c1 = c[m1 | m2]


import cProfile, pstats

profiler = cProfile.Profile()
profiler.enable()

mask = pd.DataFrame(True, index=c.index, columns=c.columns)
mask['power'] = c['power'] > 10

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats(10)


# qp.diff()

In [22]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, qpDict







df_new, df_old = get_dfs()




In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


df_new, df_old = get_dfs()

print('df_new:')
display(df_new)

print('df_old:')
display(df_old)

print('mode=new:')
display(qp.diff(df_new, df_old, mode='new'))

print('mode=new+:')
display(qp.diff(df_new, df_old, mode='new+'))

print('mode=old:')
display(qp.diff(df_new, df_old, mode='old'))

print('mode=mix:')
display(qp.diff(df_new, df_old, mode='mix'))



# excel_diff()

In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, qpDict



def _show_differences(
    df_new, df_old, mode='mix',
    summary='print',  #print, return, None
    max_cols=200, max_rows=20,
    newline='<br>', prefix_new='', prefix_old='old: ',
    verbosity=3,):
    '''
    shows differences between dataframes
    '''

    if not df_new.index.is_unique:
        log('error: index of new dataframe is not unique', 'qp.diff()', verbosity)
    if not df_old.index.is_unique:
        log('error: index of old dataframe is not unique', 'qp.diff()', verbosity)



    #prepare dataframes
    df_new = _format_df(df_new, fix_headers=False, add_metadata=True, verbosity=verbosity)
    df_old = _format_df(df_old, fix_headers=False, add_metadata=True, verbosity=verbosity)



    cols_added = df_new.columns.difference(df_old.columns)
    cols_removed = df_old.columns.difference(df_new.columns)
    cols_shared = df_new.columns.intersection(df_old.columns)

    rows_added = df_new.index.difference(df_old.index)
    rows_removed = df_old.index.difference(df_new.index)
    rows_shared = df_new.index.intersection(df_old.index)

    changes_all = {
        'cols added': len(cols_added),
        'cols removed': len(cols_removed),
        'rows added': len(rows_added),
        'rows removed': len(rows_removed),
        'vals added': 0,
        'vals removed': 0,
        'vals changed': 0
        }



    #create dfs showing the highlighted changes dependant on "show" settings
    if mode in ['new', 'new+']:
        df_diff = copy.deepcopy(df_new)
        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)

        #add metadata columns
        if mode == 'new+':
            cols_new = ['meta']
            cols_add = []
            for col in df_diff.columns:
                if not col.startswith(prefix_old) and col != 'meta':
                    cols_new.append(col)
                    cols_new.append(prefix_old + col)

                    if prefix_old + col not in df_diff.columns:
                        cols_add.append(prefix_old + col)

            df_diff = pd.concat([df_diff, pd.DataFrame('', index=df_diff.index, columns=cols_add)], axis=1)
            df_diff_style = pd.concat([df_diff_style, pd.DataFrame('font-style: italic', index=df_diff.index, columns=cols_add)], axis=1)
        
            df_diff = df_diff[cols_new]
            df_diff_style = df_diff_style[cols_new]


        df_diff_style.loc[:, cols_added] = f'background-color: {GREEN}'
        df_diff_style.loc[rows_added, :] = f'background-color: {GREEN}'

        df_diff.loc[rows_added, 'meta'] += 'added row'



    elif mode == 'old':
        df_diff = copy.deepcopy(df_old)
        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)
        
        df_diff_style.loc[:, cols_removed] = f'background-color: {RED}'
        df_diff_style.loc[rows_removed, :] = f'background-color: {RED}'

        df_diff.loc[rows_removed, 'meta'] += 'removed row'

    elif mode == 'mix':
        inds_old = df_old.index.difference(df_new.index)
        cols_old = df_old.columns.difference(df_new.columns)

        df_diff = pd.concat([df_new, df_old.loc[:, cols_old]], axis=1)
        df_diff.loc[inds_old, :] = df_old.loc[inds_old, :]

        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)

        df_diff_style.loc[:, cols_added] = f'background-color: {GREEN}'
        df_diff_style.loc[:, cols_removed] = f'background-color: {RED}'
        df_diff_style.loc[rows_added, :] = f'background-color: {GREEN}'
        df_diff_style.loc[rows_removed, :] = f'background-color: {RED}'

        df_diff.loc[rows_added, 'meta'] += 'added row'
        df_diff.loc[rows_removed, 'meta'] += 'removed row'

    else:
        log(f'error: unknown mode: {mode}', 'qp.diff()', verbosity)


    #highlight values in shared columns
    #column 0 contains metadata and is skipped
    cols_shared_no_metadata = [col for col in cols_shared if not col.startswith(prefix_old) and col != 'meta']

    df_new_isna = df_new.loc[rows_shared, cols_shared_no_metadata].isna()
    df_old_isna = df_old.loc[rows_shared, cols_shared_no_metadata].isna()
    df_new_equals_old = df_new.loc[rows_shared, cols_shared_no_metadata] == df_old.loc[rows_shared, cols_shared_no_metadata]

    df_added = df_old_isna & ~df_new_isna
    df_removed = df_new_isna & ~df_old_isna
    df_changed = ~df_new_isna & ~df_old_isna & ~df_new_equals_old

    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_added.mask(df_added, f'background-color: {GREEN_LIGHT}').where(df_added, '')
    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_removed.mask(df_removed, f'background-color: {RED_LIGHT}').where(df_removed, '')
    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_changed.mask(df_changed, f'background-color: {ORANGE_LIGHT}').where(df_changed, '')



    df_added_sum = df_added.sum(axis=1)
    df_removed_sum = df_removed.sum(axis=1)
    df_changed_sum = df_changed.sum(axis=1)

    changes_all['vals added'] += int(df_added_sum.sum())
    changes_all['vals removed'] += int(df_removed_sum.sum())
    changes_all['vals changed'] += int(df_changed_sum.sum())

    df_diff.loc[rows_shared, 'meta'] += df_added_sum.apply(lambda x: f'{newline}vals added: {x}' if x > 0 else '')
    df_diff.loc[rows_shared, 'meta'] += df_removed_sum.apply(lambda x: f'{newline}vals removed: {x}' if x > 0 else '')
    df_diff.loc[rows_shared, 'meta'] += df_changed_sum.apply(lambda x: f'{newline}vals changed: {x}' if x > 0 else '')


    if mode == 'new+':
        cols_shared_metadata = [prefix_old + col for col in cols_shared_no_metadata]
        df_all_modifications = (df_added | df_removed | df_changed)
        df_old_changed = df_old.loc[rows_shared, cols_shared_no_metadata].where(df_all_modifications, '')
        df_diff.loc[rows_shared, cols_shared_metadata] = df_old_changed.values




    if max_cols is not None and max_cols < len(df_diff.columns):
        log(f'warning: showing {max_cols} out of {len(df_diff.columns)} columns', 'qp.diff()', verbosity)
    if max_rows is not None and max_rows < len(df_diff.index):
        log(f'warning: showing {max_rows} out of {len(df_diff.index)} rows', 'qp.diff()', verbosity)

    
    

    df_diff = df_diff.iloc[:max_rows, :max_cols]
    df_diff_style = df_diff_style.iloc[:max_rows, :max_cols]

    #replace "<" and ">" with html entities to prevent them from being interpreted as html tags
    cols_no_metadata = [col for col in df_diff.columns if not col.startswith(prefix_old) and col != 'meta']
    if pd.__version__ >= '2.1.0':
        df_diff.loc[:, cols_no_metadata] = df_diff.loc[:, cols_no_metadata].map(lambda x: _try_replace_gt_lt(x))
    else:
        df_diff.loc[:, cols_no_metadata] = df_diff.loc[:, cols_no_metadata].applymap(lambda x: _try_replace_gt_lt(x))


    result = df_diff.style.apply(lambda x: _apply_style(x, df_diff_style), axis=None)
    changes_truncated = {key: val for key,val in changes_all.items() if val > 0}

    if summary == 'print':
        display(changes_truncated)
        return result
    elif summary == 'return':
        return result, changes_truncated
    else:
        return result


def _try_replace_gt_lt(x):
    if isinstance(x, str):
        return x.replace('<', '&lt;').replace('>', '&gt;')
    elif isinstance(x, type):
        return str(x).replace('<', '&lt;').replace('>', '&gt;')
    else:
        return x
    
def _apply_style(x, df_style):
    return df_style


def excel_diff(file_new='new', file_old='old', file_diff='diff',
    index_col=0, mode='new+',
    max_cols=None, max_rows=None, verbosity=3):
    '''
    shows differences between two excel files.

    specs and requs:
    - only sheets with the same name are compared
    - needs a unique column to use as index, or sequential order of records
    - index must be unique
    - index must correspond to the same "item" in both sheets

    if index_col=None:
    - uses sequential numbers as index instead of any given column
    - uniqueness is guaranteed
    - only works if the all sheets have the same "items" in the same rows
    '''

    summary = pd.DataFrame(columns=[
        'sheet',
        f'is in new file',
        f'is in old file',
        f'index (first col) is unique in new file',
        f'index (first col) is unique in old file',
        'cols added',
        'cols removed',
        'rows added',
        'rows removed',
        'vals added',
        'vals removed',
        'vals changed',
        ])
    results = {}
    

    #get names of all sheets in the excel files
    sheets_new = pd.ExcelFile(file_new).sheet_names
    sheets_old = pd.ExcelFile(file_old).sheet_names
    
    #iterate over all sheets
    for sheet in sheets_new:
        if sheet in sheets_old:
            if index_col is None:
                df_new = pd.read_excel(file_new, sheet_name=sheet)
                df_old = pd.read_excel(file_old, sheet_name=sheet)
            else:
                df_new = pd.read_excel(file_new, sheet_name=sheet, index_col=index_col)
                df_old = pd.read_excel(file_old, sheet_name=sheet, index_col=index_col)

            result, changes = _show_differences(
                df_new, df_old, mode=mode, summary='return',
                max_cols=max_cols, max_rows=max_rows, verbosity=verbosity
                )
            
            results[sheet] = result
        

            idx = len(summary)
            summary.loc[idx, 'sheet'] = sheet
            summary.loc[idx, f'is in new file'] = True
            summary.loc[idx, f'is in old file'] = True
            summary.loc[idx, f'index (first col) is unique in new file'] = df_new.index.is_unique
            summary.loc[idx, f'index (first col) is unique in old file'] = df_old.index.is_unique
            for key, val in changes.items():
                summary.loc[idx, key] = val
            
        else:
            idx = len(summary)
            summary.loc[idx, 'sheet'] = sheet
            summary.loc[idx, f'is in new file'] = True
            summary.loc[idx, f'is in old file'] = False
            summary.loc[idx, f'index (first col) is unique in new file'] = df_new.index.is_unique

    if file_diff:
        if not file_diff.endswith('.xlsx'):
            file_diff += '.xlsx'
        
        with pd.ExcelWriter(file_diff) as writer:
            summary.to_excel(writer, sheet_name='summary', index=False)
            if index_col:
                index = True
            else:
                index = False

            for sheet, result in results.items():
                result.data['meta'] = result.data['meta'].str.replace('<br>', '\n')
                result.to_excel(writer, sheet_name=sheet, index=index)

        log(f'info: differences saved to "{file_diff}"', 'qp.excel_diff()', verbosity)
        
    return summary, results



# file_new = 'archive/stats_new.xlsx'
# file_old = 'archive/stats_old.xlsx'

# summary, results = excel_diff('archive/stats_new.xlsx', 'archive/stats_old.xlsx', to_excel=True)

# new, old = qp.get_dfs()
# display(new, old)
# _show_differences(new, old, mode='new+')

new = 'archive/NET_BM Study_01_export_2024-04-11.xlsx'
old = 'archive/NET_BM Study_01_export_2024-03-15.xlsx'

a,b = excel_diff(new, old, file_diff='archive/export_diff.xlsx', index_col=0)

# pd_util

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.util import log, qpDict
from qplib.types import _date, _na






df = pd.DataFrame({
    'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
    'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
    'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
    'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
    'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
    'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
    'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
    'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
    'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
    'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
    'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
    'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
    })

df_new, df_old = qp.get_dfs()






# types

In [1]:
import pandas as pd
import numpy as np
import re




# "bashlike" wrappers

# tests

In [ ]:

#run tests in folder "tests" using pytest and create a test report
!pytest tests --html=tests/test_report.html


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


def get_df_simple():
    df = pd.DataFrame({
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    return df

def get_df_simple_tagged():
    df = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    df.index = [0, 1, 2]
    return df


def get_df():
    df = pd.DataFrame({
        'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
        'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
        'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
        'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
        'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
        'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
        'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
        'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
        'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
        'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
        'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
        'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
        })
    return df


def get_df_tagged():
    df1 = get_df()
    df2 = pd.DataFrame('', index=df1.index, columns=['meta', *df1.columns])
    df2.iloc[:, 1:] = df1.loc[:, :]
    return df2










# temp

In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

# df1 = qp.get_df()
# df2 = pd.DataFrame({
#         'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
#         'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
#         'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
#         'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
#         'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
#         'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
#         'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
#         'bp systle': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
#         'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
#         'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
#         'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
#         'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
#         })

# qp.diff(df1, df2, returns='df')

# df1.q()

df = qp.get_df()

df.qi()

C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_34740\1457612455.py:933: UserWarning: registration of accessor <class '__main__.DataFrameQuery'> under name 'q' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor('q')
C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_34740\1457612455.py:950: UserWarning: registration of accessor <class '__main__.DataFrameQueryInteractiveMode'> under name 'qi' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor('qi')


869,df was checked. no problems found,qp.pd_util._check_df,info,2024-09-13 17:37:56.279351


888,"no columns fulfill the condition in ""´c &weight "" and the previous conditions",qp.qlang._select_cols,Warning,2024-09-13 17:37:56.387659


893,row filter cannot be applied when no columns where selected,qp.qlang._select_rows,Warning,2024-09-13 17:37:56.430448


914,"no rows fulfill the condition in ""SELECT_ROWS (symbol: ""´r"" description: ""select rows fulfilling a condition)""""",qp.qlang._select_rows,Warning,2024-09-13 17:37:56.531031


,weight,diabetes
